<a href="https://colab.research.google.com/github/felipeaguirre66/RealStateScrapper/blob/main/query_real_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
query_dict = {
              
              'service':[],                 # argen_prop, mercado_libre, zona_prop
              'city':[],                    # ej: olivos
              'adress':[''],                # ej: libertador
              'property_type':[],           # casa, departamento
              'contract_type':[],           # venta, alquiler
              'rooms':[8,100],              # ej: [2,10]
              'squared_meters':[0,500],     # ej: [40,100]
              'buy_rent_currency':[],       # USD, $
              'expensas_currency':[],       # USD, $
              'buy_rent_price':[],          # ej: [10000, 90000]
              'expensas_price':[],          # ej: [10000, 90000]
              'total_price':[0,80000000]    # ej: [10000, 90000]
    }

# Imports and functions

In [5]:
# Installs for Google Sheet connection
import gspread
import google.auth
from google.colab import auth
from google.colab import files
from google.auth import default
!pip install --upgrade -q pygsheets
import pygsheets

# Authenticate in drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Other imports
!pip install unidecode
import unidecode
import pandas as pd


# Helper functions
def open_sheet(link_db):
  gc = pygsheets.client.Client(creds)
  sh = gc.open_by_url(link_db)
  worksheet = sh.worksheets()[-1]
  data = worksheet.get_all_values(returnas='matrix')
  df = pd.DataFrame(data[1:], columns=data[0])
  return df

def preprocess_df(df):
  df_raw = df.copy()
  df_raw['city'] = df_raw['city'].apply(lambda x: pp_text(x))
  df_raw['adress'] = df_raw['adress'].apply(lambda x: pp_text(x))
  df_raw['rooms'] = df_raw['rooms'].astype(float)
  df_raw['squared_meters'] = df_raw['squared_meters'].astype(float)
  df_raw['buy_rent_price'] = df_raw['buy_rent_price'].astype(float)
  df_raw['expensas_price'] = df_raw['expensas_price'].astype(float)
  df_raw['total_price'] = df_raw['total_price'].astype(float)
  return df_raw

def pp_text(text):
  return unidecode.unidecode(text.lower().strip())

def query_database(pp_df, query_dict):
  df_query = pp_df.copy()
  for key, value in query_dict.items():
    indices_coincidencia = []
    if len(value)>0:

      if type(value[0])==str: # String columns
        for query_val in value:
          indices_coincidencia.extend([i for i,x in enumerate(df_query[key].values) if pp_text(query_val) in x])
        df_query = df_query.iloc[indices_coincidencia]
      
      else: # int columns
        indices_coincidencia.extend([i for i,x in enumerate(df_query[key].values) if x >= value[0] and x <= value[1]])  
        df_query = df_query.iloc[indices_coincidencia]
  return df_query

  
# Download
def download_df(df, name):
  df_final.to_excel(name, index=False)
  files.download(name)

# Query Database

In [ ]:
link_db = 'https://docs.google.com/spreadsheets/d/1H3D2xBKoXIxh3k-gJqjXmWC-50fNUT58lXutkZhwo20/edit#gid=0'

df = open_sheet(link_db) # Read df
pp_df = preprocess_df(df) # Preprocess df
df_query = query_database(pp_df, query_dict) # Query df
df_final = df.iloc[df_query.index] # Get original values
download_df(df_final, 'archivo_final.xlsx')